In [14]:
import glob
import os
import sys

import numpy as np

sys.path.append(os.path.join(os.path.abspath(""), "../"))
from utils.raster_utils import raster_to_np
from utils.io_utils import load_json

ROI_LIST = ["1158_spring", "1868_summer", "1970_fall", "2017_winter"]
DATA_ROOT = "/data"
SENSOR = "s2"
CHANNELS = ["B1", "B2", "B3", "B4", "B5", "B6", "B7", "B8", "B8a", "B9", "B10", "B11", "B12"]

### Calculate mean and std

In [8]:
# Grab list of all subgrids
areas_ROI = {val: 0 for val in ROI_LIST}
for ROI in ROI_LIST:
    roi_folder = os.path.join(DATA_ROOT, f"ROIs{ROI}")
    areas = glob.glob(os.path.join(roi_folder,"*"))
    areas = set([int(os.path.split(path)[1].split("_")[1]) for path in areas])
    areas_ROI[ROI] = areas

means = np.zeros((len(CHANNELS)))
stds = np.zeros((len(CHANNELS)))
for ROI in ROI_LIST:
    for i, area in enumerate(areas_ROI[ROI]):
        data_path = os.path.join(DATA_ROOT, f"ROIs{ROI}", f"{SENSOR}_{area}")
        files = glob.glob(os.path.join(data_path, "*tif"))
        for j, file in enumerate(files):
            img_np = raster_to_np(file)
            image_stds = np.std(img_np, axis=(1, 2))
            image_means = np.mean(img_np, axis=(1, 2))
            means += image_means
            stds += image_stds

In [11]:
means/len(files)

array([ 633379.07771606,  530338.56699719,  490881.17267132,
        490214.08497399,  583009.3253326 ,  839018.49527371,
        961302.5563132 ,  936536.71236771, 1048830.08145605,
        339887.23487222,   10312.7715806 ,  871970.76634897,
        586737.5259709 ])

In [13]:
stds/len(files)

array([ 37440.93342533,  59662.58632252,  75616.84200858, 115031.06263879,
       113411.8632479 , 158592.02129126, 192365.33563002, 201670.73196264,
       214285.32119203,  60211.67835047,   1462.01034356, 195896.23901316,
       161060.25719065])

In [15]:
# Save as json
import json
means_dict = {band: mean for band, mean in zip(CHANNELS, means)}
stds_dict = {band: std for band, std in zip(CHANNELS, stds)}

with open(os.path.join(DATA_ROOT, "rasters_stats.json"), "w") as f:
    json.dump({"means": means_dict, "stds": stds_dict}, f)